About comprehensions, BigInts, Interfaces, Tasks

#  Fermat numbers

A [Fermat number][fermat] is a positive integer of the form


$F_n = 2 ^ {2^n} + 1$

Currently, the only known Fermat number that are also primes are $F_0$, $F_1$, $F_2$, $F_3$, and $F_4$.

In the following sections, I will implement different algorithms for prime decomposition,
and I will test them on sequences of Fermat numbers.

Julia does not promote automatically from Integer to BigInt.
Thus, I prefer to explicitely handle overflows:
if needed, I promote the argument $n$ to `BigInt`, to propagate the BigInt type everywhere.

Note that in the following cell I build the array using a **comprehension**.

In [1]:
function fermat(n::Integer)
    if n >= 6
        n = BigInt(n)
    end
    2 ^ (2^n) + 1
end

fermat(from, to) = [fermat(i) for i in from:to]

fermat(0, 6)

7-element Array{Union(Int64,BigInt),1}:
                    3
                    5
                   17
                  257
                65537
           4294967297
 18446744073709551617

# Iteration interface and Tasks: Trial Division

The most naive algorithm for integers factorization is [Trial Division][trial]: to find
the prime factors of $N$, we try to divide it by all prime numbers smaller than  $\sqrt{N}$.

To generate the prime numbers, I use the Sundaram sieve, but I exploit the iteration
interface of Julia to have a lazy implementation of it.


## Iteration Interface: a lazy implementation of Sundaram Sieve

Julia can be extended with new user defined types.
which can be integrated seamlessly into the language by implementing the
right [interfaces][interfaces].

Consider for example the _for_ loop

```
for i in obj
    do something with i
end
```

It can work with any object implementing the iteration interface, i.e. the methods
`start`, `next` and `done`.
That loop is syntactic sugar equivalent to

```
state = start(obj)
while !done(obj, state)
    i, state = next(obj, state)
end
```

To define a lazy version of the sundaram sieve, I create a new [composite type][composite-types],
containing the `is_prime` array of booleans (a bit array in this implementation), and the upper bound $ub$.

The type is parametric, since $ub$ can be `Integer` or `BigInt`.

In [2]:
immutable SundaramSieve{T}
    is_prime::BitArray
    ub::T
end

The iteration interface let me track the status of the iteration via the `state` object, which can be anything, and which is passed around by the three methods.

I use as state the last index checked by the sieve.

I start at 1, so `start` returns 1.

If $p$ is the current state, then `done` is true if there are no more primes between $p$ and $ub$. But to know that, the sieve must have run until $ub$. Thus, we maintain the sieving process one step beyond the prime returned at current iteration:
it computes until 7 but it returns 5; the next step it computes until 11 but it returns 7, and so on.

`next` receives the state, which allows to compute the prime it has to return, but it runs one step of the sieving until the next prime, or $ub$.

In [3]:
sundaram_sieve(n::Integer) = SundaramSieve(trues(n), div(n, 2)+1)

# the first state. In the state keep the last checked integer
Base.start(s::SundaramSieve) = 1

function Base.next(s::SundaramSieve, state)
    
    for i = state:s.ub
        step = i * 2 + 1
        for j=i+i*step:step:s.ub
            s.is_prime[j] = false
        end
        if s.is_prime[i]
            return (max(2, (state-1)*2+1), i+1)
        end
    end
    (max(2, (state-1)*2+1), s.ub)
end

Base.done(s::SundaramSieve, state) = state >= s.ub

done (generic function with 37 methods)

In [4]:
for i in sundaram_sieve(10)
    print("$i ")
end

collect(sundaram_sieve(10))

2 3 5 7 

4-element Array{Any,1}:
 2
 3
 5
 7

Another way of implementing lazy evaluated sequences are [coroutines][tasks].


## Coroutines
Coroutines are functions whose evaluation can be suspended and resumed.
To create a coroutine, pass your function to the `Task` function.
Your function can produce data, and return the control to the caller,
using the  `produce` function.

The coroutine execution can be resumed by using the `consume` function.
A Task implements also the iteration interface, so you can iterate over it
instead of explicitely calling `consume`.

In [5]:
function trial_division(n)
    function _f()
        if n < 2
            return
        end
        ub::Integer = ceil(sqrt(n)) + 1
    
        for prime in sundaram_sieve(ub)
            while n % prime == 0
                produce(prime)
                n = div(n, prime)
            end
        end
        if n > 1
            produce(n)
        end
    end
    Task(_f)
end

trial_division (generic function with 1 method)

In [6]:
f5 = fermat(5)
print("$f5 = 1")
for factor in trial_division(f5)
    print(" * $factor")
end

4294967297 = 1 * 641 * 6700417

Unfortunately we cannot run this algorithm for bigger $F$s, since we are not able to store the sieve.

In [7]:
collect(trial_division(fermat(6)))

LoadError: `BitArray{N}` has no method matching BitArray{N}(::BigInt)
while loading In[7], in expression starting on line 1

# Julia parallel computing: Pollard's rho algorithm

The function `pollard_rho` is almost a copy&paste from the wikipedia [pseudo-code][rho].

It is (most likely) able to find a divisor for the argument $n$. If it returns $n$, then $n$ can be prime,
or we should try with a different `g` or initial values of $x$ and $y$.

In [12]:
g(x, n) = (x^2 + 1) % n

function pollard_rho(n)
    x, y, d = 2, 2, 1
    
    while d == 1
        x = g(x, n)
        y = g(g(y, n), n)
        d = gcd(abs(x-y), n)
    end
    d, div(n, d)
end

pollard_rho (generic function with 1 method)

It is already able to do more things than our `trial_division`!

In [13]:
f6 = fermat(6)
x, y = pollard_rho(f6)
"$f6 = $x * $y"

"18446744073709551617 = 274177 * 67280421310721"

What is not able to do is returning all the factors of $n$. To do that we must simply recursively apply 
`pollard_rho` to the two factors of $n$, if they are different from $n$ and 1.

This can be parallelized, by exploiting coroutines.

## Cluster managers

To start, we need to create a bunch of coordinated logical processes. This is the role of `ClusterManager`s.
Julia provides two of them:

1.    `LocalManager`, used to add processes on a single machine
2.    `SSHManager`, to spawn processes on remote machines


Only the initial processes, called `master`, has the right of spawning new processes.

To ask to the `LocalManager` to create 2 new local processes, we use

In [14]:
#addprocs(2)

In [15]:
function pollard_rho_factorization(n)

    function get_factors(n)
        x, y = pollard_rho(n)
        _keep_factoring(x, n)
        _keep_factoring(y, n)
    end
    
    function _keep_factoring(x, n)
        if x > 3 && x != n
            @async get_factors(x)
        else
            produce(x)
        end
    end
    
    @async get_factors(n)
end

pollard_rho_factorization (generic function with 1 method)

In [17]:
for i in pollard_rho_factorization(f6)
    print("_ ")
end

[interfaces]: http://docs.julialang.org/en/release-0.4/manual/interfaces/
[composite-types]: http://docs.julialang.org/en/release-0.4/manual/types/#composite-types
[trial]: https://en.wikipedia.org/wiki/Trial_division
[tasks]: http://docs.julialang.org/en/release-0.4/manual/control-flow/#man-tasks
[rho]: https://en.wikipedia.org/wiki/Pollard%27s_rho_algorithm
[fermat]: https://en.wikipedia.org/wiki/Fermat_number